# MagmaSat+: Saturation pressures for Multi-Sample input

NOTE: Don't use spaces or special characters in your excel file!

This jupyter notebook made for Penny Wieser

## Initialize tools and packages that are required to execute this notebook.

In [1]:
from thermoengine import equilibrate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

/Users/kiacovin/.local/lib/python3.7/site-packages/rubicon/objc/ctypes_patch.py:24: UserWarning: rubicon.objc.ctypes_patch has only been tested with Python 3.4 through 3.6. The current version is sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0). Most likely things will work properly, but you may experience crashes if Python's internals have changed significantly.
  .format(sys.version_info)


## Create a MELTS v 1.2.0 instance
Rhyolite-MELTS version 1.0.2 is the default model.

In [2]:
melts = equilibrate.MELTSmodel('1.2.0')

# Import your file

In [3]:
excel_file='example_data.xlsx' #Put in the name of your file here
compo_input = pd.read_excel(excel_file)
compo_input=compo_input.set_index('Label') #Name of whatever your column header is for the sample name
compo_input.head(10)  # or insert whatever number to print how many rows you want

# compo_input.tail()    #to print the last few rows 

,SiO2,TiO2,Al2O3,Fe2O3,Cr2O3,FeO,MnO,MgO,NiO,CoO,CaO,Na2O,K2O,P2O5,H2O,CO2,Press,Temp
Label,,,,,,,,,,,,,,,,,,
BT-ex,77.50,0.0800,12.50,0.207,0,0.4730,0.0000,0.0300,0,0,0.4300,3.98,4.88,0.0,5.500000,0.050000,500,900
TVZMa-ex,78.37,0.1300,11.94,0.000,0,0.9900,0.0400,0.0500,0,0,0.5300,3.80,4.14,0.0,4.060000,0.005000,600,800
TVZOh-ex,77.90,0.0800,12.15,0.000,0,0.9500,0.0500,0.0600,0,0,0.5500,4.05,4.12,0.0,4.630000,0.005000,50,900
Oh48-FTIR1-MI1-a,78.27,0.0298,12.02,0.000,0,0.9828,0.0336,0.0515,0,0,0.4772,4.05,4.09,0.0,4.214912,0.004566,250,950
Oh48-FTIR1-MI1-b,78.27,0.0298,12.02,0.000,0,0.9828,0.0336,0.0515,0,0,0.4772,4.05,4.09,0.0,4.005816,0.004448,500,1025
Oh48-FTIR1-MI1-IRc,78.27,0.0298,12.02,0.000,0,0.9828,0.0336,0.0515,0,0,0.4772,4.05,4.09,0.0,3.885649,0.004654,5000,925
Oh50-4.1,77.91,0.0984,12.07,0.000,0,1.0556,0.0257,0.0999,0,0,0.5216,4.04,4.18,0.0,4.641843,0.004566,1000,862
Oh50-4.2,77.91,0.0984,12.07,0.000,0,1.0556,0.0257,0.0999,0,0,0.5216,4.04,4.18,0.0,4.402133,0.004448,100,770
Oh49-4.1,77.92,0.0099,12.11,0.000,0,1.0020,0.0672,0.0546,0,0,0.5346,4.01,4.30,0.0,4.283934,0.004566,1000,855


## Required input: Does your file have temp information, or will you calculate all saturation pressures at one given temp?

In [4]:
temp = 900.0 #enter a temperature here in degrees C if your file has no temperature information

file_has_temp = False #Set this to True if your file has a column with a temperature value given for each sample.
#Set to False if your file does not have temperature information, and you wish to use the numerical value supplied above.

file_temp_column_name = 'Temp' #type in the label (column header) for your column of temperatures
#Only needed if file_has_temp is True.

## Suppress phases that are not required in the simulation.

In [5]:
oxides = melts.get_oxide_names()
phases = melts.get_phase_names()

b = melts.get_phase_inclusion_status()
for phase in phases:
    melts.set_phase_inclusion_status({phase:False})

#set only Fluid and Liquid to True
melts.set_phase_inclusion_status({'Fluid':True, 'Liquid':True})

a = melts.get_phase_inclusion_status()

print("Only allowable phases:")
for key, value in a.items():
    if value == True:
        print(key)

Only allowable phases:
Fluid
Liquid


## Now python does some stuff...

In [6]:
bulk_comp = {}
startingP = []
startingP_ref = []
satP = []
flmass = []
flH2O = []
flCO2 = []
for index, row in compo_input.iterrows():
    bulk_comp = {oxide:  row[oxide] for oxide in oxides}
    feasible = melts.set_bulk_composition(bulk_comp)
    
    if file_has_temp == True:
        temp = row[file_temp_column_name]
    else:
        temp = temp
    
    fluid_mass = 0.0
    press = 1000.0
    while fluid_mass <= 0.0:
        press -= 100.0
        
        output = melts.equilibrate_tp(temp, press, initialize=True)
        (status, temp, i, xmlout) = output[0]
        fluid_mass = melts.get_mass_of_phase(xmlout, phase_name='Fluid')
        
        if press <= 0:
            break
            
    startingP.append(press+100.0)

compo_input["StartingP"] = startingP
    
for index, row in compo_input.iterrows():
    bulk_comp = {oxide:  row[oxide] for oxide in oxides}
    feasible = melts.set_bulk_composition(bulk_comp)
    
    if file_has_temp == True:
        temp = row[file_temp_column_name]
    else:
        temp = temp
    
    fluid_mass = 0.0
    press = row["StartingP"]
    while fluid_mass <= 0.0:
        press -= 10.0
        
        output = melts.equilibrate_tp(temp, press, initialize=True)
        (status, temp, i, xmlout) = output[0]
        fluid_mass = melts.get_mass_of_phase(xmlout, phase_name='Fluid')
        
        if press <= 0:
            break
            
    startingP_ref.append(press+10.0)
    
compo_input["StartingP_ref"] = startingP_ref
    
for index, row in compo_input.iterrows():
    bulk_comp = {oxide:  row[oxide] for oxide in oxides}
    feasible = melts.set_bulk_composition(bulk_comp)
    
    if file_has_temp == True:
        temp = row[file_temp_column_name]
    else:
        temp = temp
    
    fluid_mass = 0.0
    press = row["StartingP_ref"]
    while fluid_mass <= 0.0:
        press -= 1.0
        
        output = melts.equilibrate_tp(temp, press, initialize=True)
        (status, temp, i, xmlout) = output[0]
        fluid_mass = melts.get_mass_of_phase(xmlout, phase_name='Fluid')
        
        if press <= 0:
            break
            
    satP.append(press)
    flmass.append(fluid_mass)
    
    flcomp = melts.get_composition_of_phase(xmlout, phase_name='Fluid')
    flH2O.append(flcomp["H2O"])
    flCO2.append(flcomp["CO2"])
    
#     print (index)
#     print ("Pressure = " + str(press))
#     print ("Fluid mass = " + str(fluid_mass))
#     print ("\n")  


compo_input["SaturationPressure_MPa"] = satP
compo_input["FluidMassAtSaturation_grams"] = flmass
compo_input["H2Ofluid_wtper"] = flH2O
compo_input["CO2fluid_wtper"] = flCO2
del compo_input["StartingP"]
del compo_input["StartingP_ref"]

print (compo_input)
                     

                     SiO2    TiO2  Al2O3  Fe2O3  Cr2O3     FeO     MnO  \
Label                                                                    
BT-ex               77.50  0.0800  12.50  0.207      0  0.4730  0.0000   
TVZMa-ex            78.37  0.1300  11.94  0.000      0  0.9900  0.0400   
TVZOh-ex            77.90  0.0800  12.15  0.000      0  0.9500  0.0500   
Oh48-FTIR1-MI1-a    78.27  0.0298  12.02  0.000      0  0.9828  0.0336   
Oh48-FTIR1-MI1-b    78.27  0.0298  12.02  0.000      0  0.9828  0.0336   
Oh48-FTIR1-MI1-IRc  78.27  0.0298  12.02  0.000      0  0.9828  0.0336   
Oh50-4.1            77.91  0.0984  12.07  0.000      0  1.0556  0.0257   
Oh50-4.2            77.91  0.0984  12.07  0.000      0  1.0556  0.0257   
Oh49-4.1            77.92  0.0099  12.11  0.000      0  1.0020  0.0672   
Oh49-4.2            77.92  0.0099  12.11  0.000      0  1.0020  0.0672   
Ma55-5a.1           77.68  0.0096  12.27  0.000      0  1.0272  0.0628   
Ma57-3b.2           77.90  0.0498  12.

## Save our newly calculated saturation pressures and fluid masses to an excel file   

In [11]:
compo_input.to_excel("MSP_output.xlsx") 